# Multimodal Text-Caption Model (MTC)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img align="left" src="Images\text-captions.png" width="400">

In [9]:
from Utils import load_data, preprocessing, model_performances
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import json
import keras
from functools import partial
from sklearn.metrics import classification_report
import time

In [3]:
preprocessing.set_seed(1)

## Utils

In [5]:
label_column = "misogynous"
input_columns = ['text_USE', 'caption_USE']
threshold = 0.5

embed_size = 512  # 512-length array with Universal Sentence Encoder algorithm
input_shape = embed_size*2
batch_size = 64
epochs = 100

## Load Training data

In [6]:
# ________________________________________load training data ___________________________________________________
meme_df = load_data.load_azure_caption_training()

In [7]:
meme_df['text_USE'] = preprocessing.use_preprocessing(meme_df, 'Text Transcription')
meme_df['caption_USE'] = preprocessing.use_preprocessing(meme_df, 'caption')

In [8]:
meme_df.head()

,file_name,misogynous,Text Transcription,caption,text_USE,caption_USE
0,5827.jpg,0,"*Gets on Pornhub* Woman on Porn Ad: ""Are you s...","graphical user interface, website","[0.04223586246371269, -0.028678087517619133, 0...","[0.011670215055346489, -0.029457250609993935, ..."
1,2454.jpg,0,When your high school girlfriend finally turns...,text,"[-0.016387682408094406, -0.004176544025540352,...","[0.05359172075986862, -0.02190956473350525, -0..."
2,6492.jpg,1,Me every time I refuse to objectify women I ca...,"graphical user interface, text, application","[0.011072046123445034, -0.023999786004424095, ...","[0.05643597990274429, -0.03217722102999687, 0...."
3,2054.jpg,1,Verizon Q Search News r/kotakuinaction2 There ...,a screenshot of a cartoon,"[-0.02618846856057644, -0.034125760197639465, ...","[0.06304869800806046, 0.018172727897763252, 0...."
4,5388.jpg,0,me watching a horror movie NO DUDE WASN'T ME s...,a screenshot of a video game,"[-0.0059230634942650795, 0.006091502029448748,...","[0.022163882851600647, -0.004020996391773224, ..."


## Models on trainig data
Training data are split in 10Fold and used to train models. Trained models are tested on the test fold. Models take as input text embedding and the captions embeddings.
Predictions are saved in './multimodal_text_captions/predictions/text_captionss_pred_10Fold.csv' file, with the following columns:
    id: meme unique id
    real: true label
    pred: predicted value.

Model performances are saved in './Multimodal/performances/text_captions_results_10Fold.txt' file.
Models are not saved. To save models add command 'model.save("my_model")' than load them with the command
'model = keras.models.load_model("my_model")'.

In [10]:
# ________________________________________Utils ___________________________________________________
path_models = './Multimodal/models/'
path_predictions = './Multimodal/predictions/'
path_performances = './Multimodal/performances/'
file_out = path_performances + 'text_captions_results_10Fold.txt'
predictions_csv_path = path_predictions + 'text_captions_pred_10Fold.csv'

for path in [path_models, path_predictions, path_performances]:
    if not os.path.exists(path):
        os.makedirs(path)

file = open(file_out, 'a+')
file.truncate(0)  # erase file content
file.close()

In [11]:
# ________________________________________train model on training data 10Fold________________________________________
kf = KFold(n_splits=10, shuffle=False)

iteration = 0
real_values = np.array([])
predict_values = np.array([])
ids = np.array([])

In [ ]:
for train_index, test_index in kf.split(meme_df):  # split into train and test
    preprocessing.set_seed(iteration)
    x_train, y_train, x_val, y_val, x_test, y_test = preprocessing.elaborate_data_10fold(meme_df,
                                                                                        train_index,
                                                                                        test_index,
                                                                                        iteration,
                                                                                        input_columns,
                                                                                        label_column)
    model, history = model_performances.get_trained_model(x_train, 
                            y_train, 
                            x_val, 
                            y_val,
                            input_shape=input_shape, 
                            activation_function='LeakyReLU', 
                            neurons=input_shape/2, 
                            dropout=0.2, 
                            epochs=100)
    iteration = iteration + 1

    # make prediction on GS
    pred = model.predict(x_test, batch_size=batch_size)

    predict_values = np.append(predict_values, pred)
    real_values = np.append(real_values, y_test)
    ids = np.append(ids, meme_df.iloc[test_index, :]['file_name'].tolist())

    result_df = meme_df.iloc[test_index, [0, 1]]
    result_df['score_col'] = pred

    # write on file
    file = open(file_out, "a+")
    file.write('\n\nITERAZIONE ' + str(iteration) + '\n')
    file.write(json.dumps(model_performances.compute_confusion_rates(result_df, 'score_col', 'misogynous', threshold)))
    file.write('\n') 
    file.write(classification_report(result_df['misogynous'].values, (result_df['score_col']>threshold).astype(int).values, target_names=['not_mis','mis']))
    file.close()

In [13]:
# results dataframe, save predictions
result_df = pd.DataFrame({'id': ids, 'real': real_values.astype(int), 'pred': predict_values})
result_df.to_csv(predictions_csv_path, index=False, sep='\t')

# Overall metrics write on file
file = open(file_out, "a+")
file.write('\n\n10 Fold Results ' + str(iteration) + '\n')
file.write(json.dumps(model_performances.compute_confusion_rates(result_df, 'pred', 'real', threshold)))
file.write('\n') 
file.write(classification_report(result_df['real'].values, (result_df['pred']>threshold).astype(int).values, target_names=['not_mis','mis']))
file.write('\n AUC:') 
file.write(str(model_performances.compute_auc(result_df['real'].values, result_df['pred'].values)))
file.close()

In [15]:
model_performances.compute_confusion_rates(result_df, 'pred', 'real', threshold)

{'tpr': 0.8584,
 'tnr': 0.7182,
 'fpr': 0.28180000000000005,
 'fnr': 0.14159999999999995,
 'precision': 0.7528503771268199,
 'recall': 0.8584,
 'accuracy': 0.7883,
 'f1': 0.8021680216802168,
 'auc': 0.8756587}

## Load Test and Synthetic Data

In [17]:
test_df = load_data.load_azure_caption_test()
test_df['text_USE'] = preprocessing.use_preprocessing(test_df, 'Text Transcription')
test_df['caption_USE'] = preprocessing.use_preprocessing(test_df, 'caption')

x_test, y_test = preprocessing.elaborate_input(test_df, input_columns, label_column)

In [18]:
syn_df = load_data.load_azure_caption_syn()
# shuffle according to the pre-established order
syn_df = load_data.shuffle_syn(syn_df, 'file_name')

In [20]:
syn_df['text_USE'] = preprocessing.use_preprocessing(syn_df, 'Text Transcription')
syn_df['caption_USE'] = preprocessing.use_preprocessing(syn_df, 'caption')

In [21]:
res_test = test_df[['file_name', 'misogynous']].copy()
res_syn = syn_df[['file_name', 'misogynous']].copy()

## Models on Training-Test Data

### Train models on training data with a 10Fold approach
 Load training data, compute with USE embedding for text and for categories.
 Fit 10 models using whole training data, each with a different fold as validation.

In [11]:
x_syn, y_syn = preprocessing.elaborate_input(syn_df, input_columns, label_column)

In [16]:

# ________________________________________Utils ___________________________________________________
MODELNAMES = ['multimodal_text_captions_v{}'.format(i) for i in range(10)]
path_models = './Multimodal/models/Bias/'
model_name = 'multimodal_text_captions'

if not os.path.exists(path_models):
    os.makedirs(path_models)

In [ ]:
# ________________________________________train model on training data 10Fold________________________________________
kf = KFold(n_splits=10, shuffle=False)
iteration = 0

for train_index, val_index in kf.split(meme_df):
    preprocessing.set_seed(iteration)
    MODELNAME = MODELNAMES[iteration]

    x_train, y_train = preprocessing.elaborate_input(meme_df.iloc[train_index, :], input_columns, label_column)
    x_val, y_val = preprocessing.elaborate_input(meme_df.iloc[val_index, :], input_columns, label_column)

    model, history = model_performances.get_trained_model(x_train, 
                            y_train, 
                            x_val, 
                            y_val,
                            input_shape=input_shape, 
                            activation_function='LeakyReLU', 
                            neurons=input_shape/2, 
                            dropout=0.2, 
                            epochs=epochs)

    # save each model once for all
    model.save(path_models + MODELNAME)
    iteration = iteration + 1


### Load models and test on Test dataset
Load models trained by Clarifai_bias_01_TrainModels on Training data with a 10-fold approach
Load test data and compute USE embedding both for text and for clarifai tags Make prediction on test set with each model trained on training data.
Create a csv with file_name, real misogyny value and a column for each model’s predictions

In [22]:
x_syn, y_syn = preprocessing.elaborate_input(syn_df, input_columns, label_column)

In [23]:
MODELNAMES = ['multimodal_text_captions_v{}'.format(i) for i in range(10)]
path_models = './Multimodal/models/Bias/'
path_results_test = './Multimodal/predictions/predictions_test/'
path_results_syn = './Multimodal/predictions/predictions_syn/'
path_performances = './Multimodal/performances'
model_name = 'multimodal_text_captions'

file_out_test = "./Multimodal/performances/text_captions_Results_Test.txt"
file_out_syn = "./Multimodal/performances/text_captions_Results_Syn.txt"
file_out_bias = "./Multimodal/performances/text_captions_Results_Bias.txt"

for path in [path_results_test, path_results_syn, path_performances]:
    if not os.path.exists(path):
        os.makedirs(path)

for file_name in [file_out_test, file_out_syn, file_out_bias]:
    file = open(file_name, 'a+')
    file.truncate(0)  # erase file content
    file.close()

In [ ]:
# ______________________________ retrieve saved 10 models and make predictions _________________________________
kf = KFold(n_splits=10, shuffle=False)
syn_folds = kf.split(syn_df)
pred_syn=[]
syn_10_df=pd.DataFrame()
for MODELNAME in MODELNAMES:
    # LOAD MODEL
    loaded_model = keras.models.load_model(path_models + MODELNAME)

    # make prediction on test
    predict_values = loaded_model.predict(x_test, batch_size=batch_size)
    res_test[MODELNAME] = pd.DataFrame(predict_values)[0]

    # make prediction on syn
    predict_values = loaded_model.predict(x_syn, batch_size=batch_size)
    res_syn[MODELNAME] = pd.DataFrame(predict_values)[0]

    # performances on splitted Syn
    _, test_syn = next(syn_folds)
    syn_10_df['label_'+MODELNAME]=list(res_syn[label_column][test_syn].values)
    syn_10_df[MODELNAME]= list(res_syn[MODELNAME][test_syn].values)
    syn_10_df['file_name_'+MODELNAME]=list(res_syn['file_name'][test_syn].values)


res_test.to_csv(path_results_test + "baseline_" + model_name + "_scores.tsv", sep="\t", index=False)
res_syn.to_csv(path_results_syn + "baseline_" + model_name + "_SYN_scores.tsv", sep="\t", index=False)

model_performances.plot_model_family_auc(res_test, MODELNAMES, label_column)

model_performances.confusion_rates_on_file(file_out_test, res_test, MODELNAMES, label_column, threshold)
model_performances.confusion_rates_on_file(file_out_syn, res_syn, MODELNAMES, label_column, threshold)
model_performances.confusion_rates_on_file_10Fold_syn(file_out_syn, syn_10_df, MODELNAMES, threshold)



In [25]:
# __________________________________ Identity Elements __________________________________
# Load Identity Terms and Identity Tags
# NB: Only the identity terms and tags present at least in one misogynous and one non-misogynous meme are considered

Identity_Terms = load_data.read_clear_identity_terms()
Identity_Tags = load_data.read_clear_identity_tags()

res_syn = res_syn.merge(load_data.load_syn_identity_data().drop(columns=['misogynous', 'Text Transcription']),
                        how='inner', on='file_name')

In [26]:
# _________________________________Compute Bias Metrics_____________________________________________________
# Computes per-subgroup metrics for all subgroups and a list of models.
subgroups = Identity_Terms
model_performances.compute_bias_metrics_for_models(res_syn,
                                                   subgroups,
                                                   MODELNAMES,
                                                   label_column)

model_performances.bias_metrics_on_file(file_out_bias, res_test, res_syn, Identity_Terms, MODELNAMES, label_column)


In [ ]:
# _________________________________Compute Bias Metrics Multilabel_____________________________________________________

model_performances.multilabel_bias_metrics_on_file(file_out_bias, res_test, res_syn, Identity_Terms, Identity_Tags,
                                                   MODELNAMES, label_column)
